# Basic Operators

There are three basic operators out of which pretty much all other finite difference operators
can be built: the identity operator, the translation operator and difference operator.

In [1]:
from sympy import *

## Identity Operator

The simplest operator is the identity operator $I$. Applying the operator to some function $f(x)$ just returns the
function:

$$
I f(x) = f(x)
$$

Of course, we can apply the operator repeatedly:

$$
I^2 f(x) = I f(x) = f(x)
$$

So as an operator identity, we have 

$$
I^2 = I
$$

And of course, the inverse operator is also simply

$$
I^{-1} = I
$$

Let's build a representation of $I$ in Python:

In [2]:
class Identity:
        
    def __call__(self, f):
        """Applies the identity operator to a function."""
        return f
    
    def __pow__(self, *args):
        """Defines the power operation for the identity operator."""
        return self

In [3]:
f, g = symbols('f, g', cls=Function)
x, h, a, b = symbols('x, h, a, b', real=True)
m, n = symbols('m, n', integer=True)

In [4]:
I = Identity()
I(f(x))

f(x)

In [5]:
(I**2)(f(x))

f(x)

In [6]:
(I**-1)(f(x))

f(x)

## Translation Operator

Another basic operation is shifting the function from one point $x$ to the next grid point $x+h$. The corresponding operator is called *translation operator* $T$:

$$
Tf(x) = f(x+h)
$$

$$
T^2 f(x) = f(x+2h)
$$

Of course, the inverse operation is shifting into the other direction:

$$
T^{-1} f(x) = f(x-h)
$$

Note that we are shifting with respect of $x$, so if we have $f(ax)$, we get

$$
T f(ax) = f(a(x+h)) = f(ax+ah)
$$

In [7]:
class Translation:
    
    def __init__(self, shift=Symbol('h', real=True)):
        """Initializes the translation operator with an default shift to the right."""
        self.shift = shift        
    
    def __call__(self, f, var_name=Symbol('x', real=True)):
        """
        Applies the translation operator to a function.

        Parameters:
        - f: The function to be translated.
        - var_name: The variable name (default is 'x').

        Returns:
        The translated function.
        """
        if isinstance(f, Function):
            f_ = Function(f.name)
            arg = f.args[0]
            return f_(arg.subs(var_name, var_name + self.shift))
        else:
            return f.subs(var_name, var_name + self.shift)
    
    def __pow__(self, exponent):
        """
        Defines the power operation for the translation operator.

        Parameters:
        - exponent: The exponent to which the translation operator is raised.

        Returns:
        A new instance of the translation operator.
        """
        if exponent == 0:
            return Identity()
        else:
            return Translation(exponent * self.shift)

In [8]:
T = Translation()

In [9]:
T(f(x))

f(h + x)

In [10]:
(T**2)(f(x))

f(2*h + x)

In [11]:
(T**m)(f(x))

f(h*m + x)

In [12]:
(T**-m)(f(x))

f(-h*m + x)

In [13]:
T(f(a*x))

f(a*(h + x))

In [14]:
expand(_)

f(a*h + a*x)

Note that in the Python implementation, we have made the distinction if the argument given to 
the translation operator is a function or not. So this allows us to also handle cases like $T x^2 = (x+h)^2$ where no function object is involved:

In [15]:
T(x**2)

(h + x)**2

In [16]:
expand(_)

h**2 + 2*h*x + x**2

$$
T^3(3x-4)
$$

In [17]:
(T**3)(3*x-4)

9*h + 3*x - 4

## Difference Operator

The third basic operator is the difference operator $\Delta$, which is defined as

$$
\Delta f(x) = f(x+h) - f(x)
$$

Its importance stems from the definition of the derivative, of course:

$$
\frac{df}{dx} = \lim_{h\rightarrow 0} \frac{f(x+h) - f(x)}{h} = \lim_{h\rightarrow 0} \frac{\Delta f(x)}{\Delta x}
$$

Note that $\Delta$ can be written in terms of the identity operator and the translation operator:

$$
\Delta = T - I
$$

So when we implement it in Python, we can reuse the classes for $T$ and $I$. But first, we have to extend them by addition and subtraction functionality:

In [18]:
class Difference:
    
    def __call__(self, f):
        """
        Applies the difference operator to a function.

        Parameters:
        - f: The function to which the difference operator is applied.

        Returns:
        The result of the difference operator.
        """
        T = Translation()
        I = Identity()
        return T(f) - I(f)

In [19]:
Δ = Difference()

In [20]:
Δ(f(x))

-f(x) + f(h + x)

In [21]:
Δ(f(x)) / Δ(x)

(-f(x) + f(h + x))/h

Of course, we can call it repeatedly:

In [22]:
Δ(Δ(f(x)))

f(x) - 2*f(h + x) + f(2*h + x)

We could have defined the difference operator in the other direction,

$$
\Delta' f(x) = f(x) - f(x-h)
$$

but actually, it doesn't matter, since it does not give any new information. In fact, we can simply write

$$
\Delta' f(x) = T^{-1} (f(x+h) - f(x)) = T^{-1}\Delta f(x)
$$

So we have 

$$
    \Delta' = T^{-1}\Delta
$$

By the way, $\Delta$ and $T$ commute, i.e. the order in which we apply them does not matter:

$$
T \Delta = \Delta T
$$

Or, if you have a physics background, you would write it as the commutator 

$$[T, \Delta] = T \Delta - \Delta T = 0$$

Let's check with our code:

In [23]:
T(Δ(f(x)))

-f(h + x) + f(2*h + x)

In [24]:
Δ(T(f(x)))

-f(h + x) + f(2*h + x)

In [25]:
T(Δ(f(x))) - Δ(T(f(x)))

0

### The difference operator is linear

$$
\Delta \left(af(x) + bg(x)\right) = a\Delta f(x) + b \Delta g(x)
$$

In [26]:
Δ(a*f(x) + b*g(x))

-a*f(x) + a*f(h + x) - b*g(x) + b*g(h + x)

In [27]:
a*Δ(f(x)) + b*Δ(g(x))

a*(-f(x) + f(h + x)) + b*(-g(x) + g(h + x))

In [28]:
_ - __

a*(-f(x) + f(h + x)) + a*f(x) - a*f(h + x) + b*(-g(x) + g(h + x)) + b*g(x) - b*g(h + x)

In [29]:
simplify(_)

0

### Product Rule

In contrast to ordinary calculus, the product rule in finite difference calculus is

$$
\Delta (f(x) g(x)) = g(x) \Delta f(x) + f(x)\Delta g(x) + \Delta f(x) \Delta g(x)
$$

In [30]:
Δ(f(x) * g(x))

-f(x)*g(x) + f(h + x)*g(h + x)

In [31]:
expand(Δ(f(x)) * g(x) + Δ(g(x)) * f(x) + Δ(f(x)) * Δ(g(x)))

-f(x)*g(x) + f(h + x)*g(h + x)

In [32]:
_ - __

0

### Division Rule

The division rule is also a bit different from ordinary calculus. It reads

$$
\Delta\frac{f(x)}{g(x)} = \frac{g(x)\Delta f(x)- f(x)\Delta g(x)}{g(x)g(x+h)}
$$

In [33]:
Δ(f(x) / g(x))

-f(x)/g(x) + f(h + x)/g(h + x)

In [34]:
(g(x) * Δ(f(x)) - f(x) * Δ(g(x))) / (g(x)*g(x+h))

((-f(x) + f(h + x))*g(x) - (-g(x) + g(h + x))*f(x))/(g(x)*g(h + x))

In [35]:
__ - _

-((-f(x) + f(h + x))*g(x) - (-g(x) + g(h + x))*f(x))/(g(x)*g(h + x)) - f(x)/g(x) + f(h + x)/g(h + x)

In [36]:
simplify(_)

0

## Derivative Operator in Finite Differences

From ordinary calculus, we know Taylor's theorem, which allows us to express a function
as

```{math}
:label: taylor
f(x+h) = f(x) + \frac{h}{1!}\frac{df(x)}{dx} + \frac{h^2}{2!}\frac{d^2f(x)}{dx^2} + \frac{h^3}{3!}\frac{d^3f(x)}{dx^3} + \dots
```


From here it is easy to obtain an operator identity connecting the exact derivative operator with the difference operator. We can write the right-hand side of {eq}`taylor` as

```{math}
\exp\left(h \frac{d}{dx}\right) f(x)
```

and the left hand-side as

$$
f(x+h) = T f(x)
$$

So we have the operator identity

$$
T = \exp\left(h \frac{d}{dx}\right)
$$

On the other hand, we know that $\Delta = T - I$, so we have

$$
I + \Delta = \exp\left(h \frac{d}{dx}\right)
$$

Taking the logarithm, we get

$$
\frac{d}{dx} = \frac{1}{h} \ln\left(I + \Delta\right)
$$

Note that this expression is still exact! But how do we evaluate this? Well, simply expand the logarithm as a power series.

$$
\frac{d}{dx} = \frac{1}{h} \left(\frac{\Delta}{1} - \frac{\Delta^2}{2} + \frac{\Delta^3}{3} - \frac{\Delta^4}{4} + \dots \right)
$$

Can't remember the power series of the logarithm? No problem, *SymPy* helps you out:

In [37]:
series(log(1+x), x)

x - x**2/2 + x**3/3 - x**4/4 + x**5/5 + O(x**6)